In [1]:
import pandas as pd
import json
from environs import Env
from graph_functions import query_to_df, start_graph_session_crashdown_w_auth, is_it_cancer, is_it_breast_cancer
from IPython.display import Image
import requests

env = Env()
#session = start_graph_session(env) 
session = start_graph_session_crashdown_w_auth(env) 

38687


### Extract genes expressed in breast tissue

In [24]:

query = """
MATCH  (gene:Gene)-[gene_to_tissue:EXPRESSED_IN]->(tissue:Tissue) 
WHERE gene_to_tissue.tpm > 1
AND  tissue.name = 'Breast - Mammary Tissue'
RETURN gene.name, gene_to_tissue.tpm as TPM
ORDER BY TPM desc
"""

df = query_to_df(session, query)

df




MATCH  (gene:Gene)-[gene_to_tissue:EXPRESSED_IN]->(tissue:Tissue) 
WHERE gene_to_tissue.tpm > 1
AND  tissue.name = 'Breast - Mammary Tissue'
RETURN gene.name, gene_to_tissue.tpm as TPM
ORDER BY TPM desc



,gene.name,TPM
0,MT-ND4,32170.00000
1,MT-CO2,28380.00000
2,FTL,5355.00000
3,FABP4,4585.00000
4,RPS18,3804.00000
...,...,...
16907,RP11-800A3.4,1.00245
16908,OAZ3,1.00150
16909,RP11-539I5.1,1.00150
16910,RP11-757O6.1,1.00110


In [25]:
# add expression level categories
df['tpm_category'] = ''

for ind in df.index:
    if df['TPM'][ind] >= 100:
        df['tpm_category'][ind] = "high"
    elif df['TPM'][ind] < 100 and df['TPM'][ind] > 10 :
        df['tpm_category'][ind] = "medium"
    elif df['TPM'][ind] <= 10:
        df['tpm_category'][ind] = "low"
        
df['tpm_category'].values_count()

/Users/ny19205/opt/anaconda3/envs/neo4j_build/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ny19205/opt/anaconda3/envs/neo4j_build/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ny19205/opt/anaconda3/envs/neo4j_build/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #

AttributeError: 'Series' object has no attribute 'values_count'

In [14]:
df['tpm_category'].value_counts()

medium    8039
low       7918
high       955
Name: tpm_category, dtype: int64

In [15]:
# get drugs and diseases
gene_list1 = list(set(df['gene.name']))

query="""
match (gene:Gene)-[]-(drug:Drug)-[]-(d:Disease)
where gene.name in {}
return gene.name , drug.label, d.label
""".format(gene_list1)

exp_to_disease = query_to_df(session, query, False)
exp_to_disease

,gene.name,drug.label,d.label
0,NDUFB10,METFORMIN,acanthosis nigricans (disease)
1,NDUFB10,METFORMIN,altitude sickness
2,EPHX1,CARBAMAZEPINE,trigeminal neuralgia
3,NDUFS5,METFORMIN,acanthosis nigricans (disease)
4,NDUFS5,METFORMIN,altitude sickness
...,...,...,...
613,CDK7,ZOTIRACICLIB,gliosarcoma
614,CDK7,ZOTIRACICLIB,rectal cancer
615,CDK7,SELICICLIB,ACTH-dependent Cushing syndrome
616,CDK7,AT-7519,mantle cell lymphoma


In [4]:
# genes and diseases they target
exp_to_disease2 = exp_to_disease[['gene.name', 'd.label']].drop_duplicates()
exp_to_disease2

,gene.name,d.label
0,SSTR1,neuroendocrine neoplasm
1,SSTR1,ACTH-dependent Cushing syndrome
2,SSTR1,acromegaly
4,FOLR1,endometrial clear cell adenocarcinoma
5,FOLR1,endometrial serous adenocarcinoma
...,...,...
613,ADRA2A,ocular hypertension
614,FN1,soft tissue sarcoma
615,ITGAL,dry eye syndrome
616,NDUFS8,acanthosis nigricans (disease)


In [32]:
# is it cancer
exp_to_disease2 = is_it_cancer(exp_to_disease2,'d.label' )
exp_to_disease2 = is_it_breast_cancer(exp_to_disease2,'d.label' )
exp_to_disease2
exp_to_disease2.to_csv("../query_results/test.tsv", sep="\t", index=True)

#out = pd.DataFrame(pd.Series(exp_to_disease2.groupby(["gene.name", "is_it_cancer"]).size()))
#out
#out.to_csv("../query_results/test.tsv", sep="\t", index=True)


### GWAS to genes

In [27]:
query = """
MATCH (gwas:Gwas) -[gwas_to_variant:GWAS_TO_VARIANT]->(variant:Variant)-[variant_to_gene:VARIANT_TO_GENE]-> (gene:Gene)
WHERE gwas.id = '{gwas_id}' 
AND gene.name is not null 
RETURN  variant.name, gene.name

""".format(gwas_id='ieu-a-1126')

df = query_to_df(session, query)
df


MATCH (gwas:Gwas) -[gwas_to_variant:GWAS_TO_VARIANT]->(variant:Variant)-[variant_to_gene:VARIANT_TO_GENE]-> (gene:Gene)
WHERE gwas.id = 'ieu-a-1126' 
AND gene.name is not null 
RETURN  variant.name, gene.name




,variant.name,gene.name
0,rs532436,ABO
1,rs28512361,WI2-85898F10.1
2,rs5995875,MKL1
3,rs5995875,COX6B1P3
4,rs4820318,PLA2G6
...,...,...
132,rs1230666,MAGI3
133,rs1707302,PIK3R3
134,rs1707302,RP4-533D7.5
135,rs2992756,KLHDC7A


In [22]:
# get drugs and diseases
gene_list2 = list(set(df['gene.name']))

query="""
match (gene:Gene)-[]-(drug:Drug)-[]-(d:Disease)
where gene.name in {}
return gene.name , drug.label, d.label
""".format(gene_list2)

exp_to_disease = query_to_df(session, query, True)
exp_to_disease


match (gene:Gene)-[]-(drug:Drug)-[]-(d:Disease)
where gene.name in ['PLA2G6', 'AC092594.1', 'TTC28', 'PRC1', 'KRTCAP2', 'ITPR1', 'KLHDC7A', 'AQP4-AS1', 'HSCB', 'COX6B1P3', 'CHEK2', 'CUX1', 'HSPA4', 'SETBP1', 'MLLT10', 'VGLL3', 'Y_RNA', 'BRCA2', 'RPL30P5', 'TGFBR2', 'TET2', 'GRHL1', 'GATAD2A', 'MIR642A', 'ZMIZ1', 'GAREM', 'RAD51B', 'RP4-533D7.5', 'OTUD7B', 'PIDD', 'AMFR', 'OR2A1-AS1', 'GIPR', 'ATG10', 'RP11-201K10.3', 'ADSSL1', 'NREP', 'RP11-408H1.3', 'RNU1-58P', 'RCCD1', 'ASTN2', 'RP11-266N13.2', 'ARHGEF5', 'AC090453.1', 'RP11-127H5.1', 'KCNN4', 'LMX1B', 'FAM175A', 'MIER3', 'MCM8', 'PIK3R3', 'TRIM46', 'TLR1', 'CMSS1', 'TCF7L2', 'SLC25A21', 'ESR1', 'CASC8', 'PHLDA3', 'ATXN7', 'DNAJC1', 'SLC14A2', 'NEK10', 'CDCA7L', 'L3MBTL3', 'MAGI3', 'CHST9', 'ZNF365', 'RP11-134G8.8', 'RIN3', 'PDE4D', 'SLC6A18', 'AC104801.1', 'MTMR11', 'DIRC3', 'KANSL1', 'CBX8', 'RP11-438P9.2', 'ZFPM2', 'TERT', 'AC016644.1', 'SSBP4', 'EXO1', 'PRC1-AS1', 'CASC9', 'AC010969.1', 'AC012363.13', 'RANBP9', 'PEX14', 'MKL1', 

,gene.name,drug.label,d.label
0,KCNN4,CHLORZOXAZONE,"osteoarthritis, hip"
1,PIK3R3,DS-7423,endometrial cancer
2,PIK3R3,SAMOTOLISIB,endometrial cancer
3,PIK3R3,PILARALISIB,endometrial cancer
4,PDE4D,PENTOXIFYLLINE,lytic metastatic bone lesion
5,PDE4D,APREMILAST,parapsoriasis
6,CASP8,EMRICASAN,non-alcoholic steatohepatitis
7,CASP8,NIVOCASAN,non-alcoholic steatohepatitis
8,FGFR2,DERAZANTINIB,intrahepatic cholangiocarcinoma


In [37]:
# genes and diseases they target
exp_to_disease2 = exp_to_disease[['gene.name', 'd.label']].drop_duplicates()
exp_to_disease2

,gene.name,d.label
0,KCNN4,"osteoarthritis, hip"
1,PDE4D,lytic metastatic bone lesion
2,PDE4D,parapsoriasis
3,CASP8,non-alcoholic steatohepatitis
5,PIK3R3,endometrial cancer
8,FGFR2,intrahepatic cholangiocarcinoma


### test clinvar connections

In [5]:
query="""
match (gene:Gene)-[]-(d:Disease)
where gene.name in {}
return gene.name , d.label
""".format(gene_list1) # exp genes

y= query_to_df(session, query, False)
y

,gene.name,d.label
0,DHX38,retinitis pigmentosa 84
1,MASP2,immunodeficiency due to MASP-2 deficiency
2,TP63,orofacial cleft
3,TP63,orofacial cleft 8
4,TP63,limb-mammary syndrome
...,...,...
5669,WIPI2,intellectual developmental disorder with short...
5670,FH,hereditary leiomyomatosis and renal cell cancer
5671,NDUFS8,"mitochondrial complex 1 deficiency, nuclear ty..."
5672,TBC1D23,"pontocerebellar hypoplasia, type 11"


In [7]:
y = is_it_cancer(y,'d.label' )
y = is_it_breast_cancer(y,'d.label' )
y[y['is_it_cancer']=='yes']
y.to_csv("../query_results/test.tsv", sep="\t", index=True)

In [11]:
#gwas vars
query="""
match (gene:Gene)-[]-(d:Disease)
where gene.name in {}
return gene.name , d.label
""".format(gene_list2)# GWAS gens

x= query_to_df(session, query, False)
x

,gene.name,d.label
0,PLA2G6,PLA2G6-associated neurodegeneration
1,PLA2G6,neurodegeneration with brain iron accumulation
2,PLA2G6,autosomal recessive Parkinson disease 14
3,PLA2G6,Parkinson disease
4,ITPR1,aniridia-cerebellar ataxia-intellectual disabi...
...,...,...
88,CCDC88C,Hydrocephalus
89,CCDC88C,congenital hydrocephalus
90,CCDC88C,"hydrocephalus, nonsyndromic, autosomal recessi..."
91,CCDC88C,spinocerebellar ataxia type 40


In [12]:
x = is_it_cancer(x,'d.label' )
x = is_it_breast_cancer(x,'d.label' )
x[x['is_it_cancer']=='yes']

,gene.name,d.label,is_it_cancer,is_it_breast_cancer
7,CHEK2,breast neoplasm,yes,yes
10,CHEK2,prostate cancer,yes,no
11,CHEK2,hereditary breast carcinoma,yes,yes
15,MLLT10,acute myeloid leukemia,yes,no
16,BRCA2,breast neoplasm,yes,yes
19,BRCA2,kidney Wilms tumor,yes,no
22,BRCA2,Wilms tumor 1,yes,no
24,BRCA2,hereditary breast ovarian cancer syndrome,yes,yes
25,BRCA2,prostate cancer,yes,no
26,BRCA2,hereditary breast carcinoma,yes,yes


### Which of breast cancer clivar genes also occur in GWAS/expr data? 

In [2]:
query="""
match (d:Disease)
where toLower(d.label) contains "breast"
return  d.label, d.id
"""
x= query_to_df(session, query, False)
y = is_it_cancer(x,'d.label' )
y['is_it_cancer'].value_counts()
#yes    114
#no      57

y_cancer = y[y['is_it_cancer']=='yes']
y_cancer




,d.label,d.id,is_it_cancer
0,malignant breast melanoma,http://purl.obolibrary.org/obo/MONDO_0002975,yes
1,breast rhabdomyosarcoma,http://purl.obolibrary.org/obo/MONDO_0002859,yes
2,breast mucinous carcinoma,http://purl.obolibrary.org/obo/MONDO_0002707,yes
3,breast mucinous cystadenocarcinoma,http://purl.obolibrary.org/obo/MONDO_0002705,yes
5,signet ring cell breast carcinoma,http://purl.obolibrary.org/obo/MONDO_0002671,yes
...,...,...,...
165,"breast-ovarian cancer, familial, susceptibilit...",http://purl.obolibrary.org/obo/MONDO_0011450,yes
167,triple-negative breast carcinoma,http://purl.obolibrary.org/obo/MONDO_0005494,yes
168,female breast central part cancer,http://purl.obolibrary.org/obo/MONDO_0001805,yes
169,female breast lower-outer quadrant cancer,http://purl.obolibrary.org/obo/MONDO_0001850,yes


In [19]:
# map diseases to genes
bc_ids = list(set(y_cancer['d.id']))
query="""
match (gene:Gene)-[]-(d:Disease)
where d.id in {}
return  d.label, d.id,gene.name 
""".format(bc_ids) 
x= query_to_df(session, query, False)
x

print("known clinical var genes")
clinvar_bc_genes = list(set(x['gene.name']))
clinvar_bc_genes##27

known clinical var genes


['CASP8',
 'BRCA1',
 'CHEK2',
 'PPM1D',
 'RAD54L',
 'RAD51C',
 'NQO2',
 'ATM',
 'BARD1',
 'SLC22A18',
 'RAD51',
 'PTEN',
 'XRCC3',
 'AKT1',
 'RB1CC1',
 'BRCA2',
 'TP53',
 'BRIP1',
 'KRAS',
 'HMMR',
 'CDH1',
 'PHB',
 'RAD51D',
 'STK11',
 'ESR1',
 'PALB2',
 'PIK3CA']

In [10]:
# check which of these genes are drug targets

query="""
match (gene:Gene)-[]-(drug:Drug)
where gene.name in {}
return  gene.name , collect(drug.label) as drugs
""".format(clinvar_bc_genes) 
x= query_to_df(session, query, False)
x #- of those, 5 are drug targets

,gene.name,drugs
0,CASP8,"[EMRICASAN, NIVOCASAN]"
1,CHEK2,"[XL-844, PREXASERTIB]"
2,AKT1,"[MSC-2363318A, XL-418, BAY-1125976, GSK-690693..."
3,ESR1,"[ACOLBIFENE, GTX-758, ESTRONE, SR16234, ESTROG..."
4,PIK3CA,"[WX-037, PA-799, BGT-226, BAY-1082439, PKI-179..."


In [13]:
# repeat bove just extarcting drugs ----- missing lots of link to BC
query="""
match (gene:Gene)-[]-(drug:Drug)-[]-(d:Disease)
where gene.name in {}
return distinct  drug.label as drugs, d.label
""".format(clinvar_bc_genes) 
x= query_to_df(session, query, False)
x # there are 70 drugs

,drugs,d.label
0,EMRICASAN,non-alcoholic steatohepatitis
1,NIVOCASAN,non-alcoholic steatohepatitis
2,AFURESERTIB,Langerhans cell histiocytosis
3,DS-7423,endometrial cancer
4,PILARALISIB,endometrial cancer
5,SAMOTOLISIB,endometrial cancer


In [26]:
# compare with expression genes
df # from first chunk

df[df['gene.name'].isin(clinvar_bc_genes)] # 22 /27


,gene.name,TPM,tpm_category
744,PHB,124.4500,high
1870,AKT1,59.0550,medium
2673,STK11,44.1250,medium
3189,PTEN,38.2800,medium
3632,TP53,33.8150,medium
3914,NQO2,31.4550,medium
4081,CDH1,30.1350,medium
5524,RB1CC1,21.5400,medium
6953,PPM1D,15.9150,medium
7223,PIK3CA,14.9700,medium


In [28]:
# compare with GWAS genes
df # from second chunk

df[df['gene.name'].isin(clinvar_bc_genes)] # 4 /27



,variant.name,gene.name
5,rs5997389,CHEK2
43,rs11571833,BRCA2
79,rs2747652,ESR1
80,rs76956704,ESR1
113,rs3769821,CASP8
